* **Comment utiliser la conversion docx -> pdf**

1. Faire la transposition vers un dossier **DIR_TO_CONVERT** contenant les docx enrichis (nouvelles données + précédents commentaires)
2. Lancer les cellules suivantes. Les pdfs seront dans le dossier **OUTPUT_DIR**

In [ ]:
import os
import pandas as pd

In [ ]:
# Définition et création des dossiers
DIR_TO_CONVERT = "./reports_word/transposed_reports"
OUTPUT_DIR = './reports_pdf'

def mkdir_ifnotexist(path) :
    if not os.path.isdir(path) :
        os.mkdir(path)
        
mkdir_ifnotexist(OUTPUT_DIR)

In [ ]:
# Conversion docx -> pdf
files_to_convert = [os.path.join(DIR_TO_CONVERT, basename) for basename in os.listdir(DIR_TO_CONVERT)]
for filename in files_to_convert:
    !echo -----------Converting {filename}--------------
    !libreoffice --headless -convert-to pdf --outdir "{OUTPUT_DIR}" "{filename}"


In [ ]:
# Vérifier qu'on a bien les 109 départements
output_files = os.listdir(OUTPUT_DIR)
hit_dep_names = set()
for output_file in output_files:
    dep_name = output_file.split('.')[0].split('_')[-1]
    hit_dep_names.add(dep_name)


taxo_dep_df = pd.read_csv('refs/taxo_deps.csv', dtype={'dep':str, 'reg':str})
ref_dep_names = set(taxo_dep_df['libelle'].unique())
ref_dep_names.remove('Etranger')

# On regarde si tous les départements sont bien dans le dossier
assert ref_dep_names.issubset(hit_dep_names), f'Missing {ref_dep_names - hit_dep_names}'
assert len(hit_dep_names) == 109